In [101]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings("ignore")

In [102]:
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input,Activation
from keras.layers import LSTM,SimpleRNN
from keras import optimizers

In [103]:
NUM_FEATURES = 40
NUM_CLASSES = 23

In [104]:
def return_classes(df,need=False):
    le = LabelEncoder()
    le.fit(df)
    classes = le.classes_.tolist()
    if need:
        d={}
        for i,clas in enumerate(le.classes_.tolist()):
            c = le.transform([clas])
            d[i]=list(le.inverse_transform(c))[0]
        return le.transform(df),d
    return le.transform(df)

In [105]:
data = pd.read_csv("KDDCUP_modified.csv",usecols=None)
test_data = pd.read_csv("kddcup.testdata.unlabeled_10_percent",usecols=None)

In [106]:
data.columns = ['Index',
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'

]
test_data.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',

]

In [107]:
data.head()


,Index,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,395613,0,tcp,http,SF,54540,8314,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,back.
1,395614,0,tcp,http,SF,54540,8314,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,back.
2,395615,0,tcp,http,SF,54540,8314,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,back.
3,395616,0,tcp,http,SF,54540,8314,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,back.
4,395617,0,tcp,http,SF,54540,8314,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,back.


In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5443 entries, 0 to 5442
Data columns (total 43 columns):
Index                          5443 non-null int64
duration                       5443 non-null int64
protocol_type                  5443 non-null object
service                        5443 non-null object
flag                           5443 non-null object
src_bytes                      5443 non-null int64
dst_bytes                      5443 non-null int64
land                           5443 non-null int64
wrong_fragment                 5443 non-null int64
urgent                         5443 non-null int64
hot                            5443 non-null int64
num_failed_logins              5443 non-null int64
logged_in                      5443 non-null int64
num_compromised                5443 non-null int64
root_shell                     5443 non-null int64
su_attempted                   5443 non-null int64
num_root                       5443 non-null int64
num_file_creations     

In [109]:
data.shape

(5443, 43)

In [110]:
test_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
1,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
2,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
3,0,udp,private,SF,105,146,0,0,0,0,...,255,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0
4,0,udp,private,SF,105,146,0,0,0,0,...,255,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0


In [111]:
data['protocol_type'] = return_classes(data['protocol_type'])
data['service'] = return_classes(data['service'])
data['flag'] = return_classes(data['flag'])

In [112]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311028 entries, 0 to 311027
Data columns (total 41 columns):
duration                       311028 non-null int64
protocol_type                  311028 non-null object
service                        311028 non-null object
flag                           311028 non-null object
src_bytes                      311028 non-null int64
dst_bytes                      311028 non-null int64
land                           311028 non-null int64
wrong_fragment                 311028 non-null int64
urgent                         311028 non-null int64
hot                            311028 non-null int64
num_failed_logins              311028 non-null int64
logged_in                      311028 non-null int64
num_compromised                311028 non-null int64
root_shell                     311028 non-null int64
su_attempted                   311028 non-null int64
num_root                       311028 non-null int64
num_file_creations             311028 

In [113]:
data['outcome'],mapping = return_classes(data['outcome'],True)
print(mapping)

{0: 'back.', 1: 'buffer_overflow.', 2: 'ftp_write.', 3: 'guess_passwd.', 4: 'imap.', 5: 'ipsweep.', 6: 'land.', 7: 'loadmodule.', 8: 'multihop.', 9: 'neptune.', 10: 'nmap.', 11: 'normal.', 12: 'perl.', 13: 'phf.', 14: 'pod.', 15: 'portsweep.', 16: 'rootkit.', 17: 'satan.', 18: 'smurf.', 19: 'spy.', 20: 'teardrop.', 21: 'warezclient.', 22: 'warezmaster.'}


In [114]:
test_data['protocol_type'] = return_classes(test_data['protocol_type'])
test_data['service'] = return_classes(test_data['service'])
test_data['flag'] = return_classes(test_data['flag'])
test_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,2,46,9,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
1,0,2,46,9,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
2,0,2,46,9,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
3,0,2,46,9,105,146,0,0,0,0,...,255,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0
4,0,2,46,9,105,146,0,0,0,0,...,255,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0


In [115]:
X_data = data.iloc[:,1:-2].values
y_data = data.iloc[:,-1].values
X_testdata = test_data.iloc[:,:-1].values

In [116]:
X_data.shape

(5443, 40)

In [117]:
X_testdata.shape


(311028, 40)

In [118]:
X_train,X_val,y_train,y_val = train_test_split(X_data,y_data,test_size=0.2)
print(X_train.shape)

(4354, 40)


In [119]:
print(y_val.shape,y_train.shape,y_data.shape)

(1089,) (4354,) (5443,)


In [120]:
Y_val=y_val
Y_train=y_train
print(Y_val)

[15 21 20 ... 11 20 10]


In [121]:
Y_train =np_utils. to_categorical(y_train)
Y_val = np_utils.to_categorical(y_val)
#print(Y_val[0],Y_train[0])



In [122]:
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
print(X_train)

scaler = Normalizer().fit(X_val)
X_val = scaler.transform(X_val)

[[0.00000000e+00 6.94699280e-03 1.31992863e-01 ... 6.94699280e-05
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.47365420e-03 1.69998860e-01 ... 4.47365420e-03
  4.47365420e-03 0.00000000e+00]
 [0.00000000e+00 2.65277613e-03 1.00805493e-01 ... 2.12222091e-04
  0.00000000e+00 2.30791524e-03]
 ...
 [0.00000000e+00 0.00000000e+00 9.43207930e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.82962539e-04 1.94877585e-07 3.11804136e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.47367247e-02 3.69204847e-04 9.59932601e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [123]:
scaler = Normalizer().fit(X_testdata)
X_testdata = scaler.transform(X_testdata)

In [124]:
X_train = np.reshape( X_train, ( X_train.shape[0], 1,  X_train.shape[1]))
X_val = np.reshape( X_val , ( X_val .shape[0], 1,  X_val .shape[1]))
X_testdata = np.reshape( X_testdata, ( X_testdata.shape[0], 1,  X_testdata.shape[1]))
#X_val = np.reshape( X_val , ( X_val .shape[0], 1,  X_val .shape[1]))

In [125]:
# TIMESTEPS = 50
# rem = X_train.shape[0]%TIMESTEPS
# X_t = X_train[:-rem,:]
# X = X_t.reshape(X_t.shape[0]//TIMESTEPS,TIMESTEPS,X_t.shape[1])
# X.shape

In [126]:
# rem = Y_train.shape[0]%TIMESTEPS
# Y_t = Y_train[:-rem,:]
# y = Y_t.reshape(Y_t.shape[0]//TIMESTEPS,TIMESTEPS,Y_t.shape[1])
# y.shape

In [127]:
# rem = X_val.shape[0]%TIMESTEPS
# X_v = X_val[:-rem,:]
# X_v = X_v.reshape(X_v.shape[0]//TIMESTEPS,TIMESTEPS,X_v.shape[1])
# X_v.shape

In [128]:
# rem = Y_val.shape[0]%TIMESTEPS
# Y_v = Y_val[:-rem,:]
# y_v = Y_v.reshape(Y_v.shape[0]//TIMESTEPS,TIMESTEPS,Y_v.shape[1])
# y_v.shape

In [129]:
from keras.layers import Flatten
model = Sequential()
model.add(LSTM(80, input_shape=(1,40),return_sequences=True))
#model.add(TimeDistributed(Dense(1)))
#model.add(AveragePooling1D())

model.add(Flatten())
model.add(Dense(23, activation='softmax'))

In [130]:
model2 = Sequential()
model2.add(LSTM(256, input_shape=(1,40),return_sequences=True))
#model.add(TimeDistributed(Dense(1)))
#model.add(AveragePooling1D())

model2.add(Flatten())
model2.add(Dense(23, activation='softmax'))

In [131]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1, 256)            304128    
_________________________________________________________________
flatten_6 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 23)                5911      
Total params: 310,039
Trainable params: 310,039
Non-trainable params: 0
_________________________________________________________________


In [132]:
X_val.shape

(1089, 1, 40)

In [133]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1, 80)             38720     
_________________________________________________________________
flatten_5 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 23)                1863      
Total params: 40,583
Trainable params: 40,583
Non-trainable params: 0
_________________________________________________________________


In [134]:
from keras import metrics
sgd = optimizers.SGD(lr=0.01)
#model2.compile(loss='mean_squared_error',optimizer='sgd', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [135]:
model2.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

In [136]:
model2.fit(X_train,Y_train,batch_size=50,epochs=2,validation_data=(X_val,Y_val))

Train on 4354 samples, validate on 1089 samples
Epoch 1/2
4354/4354 [==============================] - 3s - loss: 2.6490 - acc: 0.4672 - val_loss: 2.1374 - val_acc: 0.6612
Epoch 2/2
4354/4354 [==============================] - 2s - loss: 1.8169 - acc: 0.6181 - val_loss: 1.5208 - val_acc: 0.6033


In [137]:
# model.fit(X_train,y_train,batch_size=50,epochs=100,validation_data=(X_val,y_val))

In [138]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import average_precision_score
predictions = model2.predict(X_val)
#y_val = np.argmax(y_val, axis=-1)
predictions = np.argmax(predictions, axis=-1)
# print(predictions.shape,Y_val.shape)
print(np.unique(y_val),(np.unique(predictions)))
length=len(np.unique(predictions))
print("lenght",length)

precision, recall, fscore, support = score(y_val,predictions)

print(recall)
#print((precision))
# print(fscore)
# precision=np.array(precision).mean()
# recall=np.array(recall).mean()
# fscore= np.array(fscore).mean()
# support=np.array(support).mean()
# print("precision",precision)
# print("fscore",fscore)
# efficiency=precision/fscore
# print(efficiency)


    

[ 0  1  2  3  4  5  6  7  8  9 10 11 14 15 17 18 20 21 22] [ 0  5 10 11 15 17 18 20 21]
lenght 9
[1.         0.         0.         0.         0.         0.91397849
 0.         0.         0.         0.         0.88172043 0.99
 0.         0.20535714 0.80612245 0.98113208 0.77669903 0.02941176
 0.        ]


In [93]:
pred = model2.predict(X_testdata)
#(pred.shape)
DoS=['back.', 'land.', 'neptune.', 'pod.', 'smurf.', 'teardrop.']
R2L=['ftp-write.', 'guess-passwd.', 'imap.', 'multihop.', 'phf.',
'spy.', 'warezclient.', 'warezmaster.']
U2R =['buffer-overflow.', 'loadmodule.', 'perl.', 'rootkit.']
Probe= ['ipsweep.', 'nmap.', 'portsweep.', 'satan.']

pred=np.argmax(pred,axis=-1)
print(np.unique(pred))
print(pred.shape)
l=len(pred)
#l=1000
print(mapping[pred[4]])
M=[None]*l
for i in range(l):
    M[i] = mapping[pred[i]]
np.unique(M)
for i in range(500):
    print(M[i])
    if(M[i] in DoS):
        print("Category is DoS:",M[i])
    elif(M[i] in R2L):
        print("Category is R2L",M[i])
    elif(M[i] in U2R):
        print("Category is U2R",M[i])
    elif(M[i] in Probe):
        print("Category is Probe",M[i])
    else:
        print("Normal")

    



[ 0  5  9 10 11 17 18 20 21]
(311028,)
normal.
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
smurf.
Category is DoS: smurf.
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
back.
Category is DoS: back.
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
back.
Category is DoS: back.
normal.
Normal
back.
Category is DoS: back.
back.
Category is DoS: back.
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
warezclient.
Category is R2L warezclient.
normal.
Normal
normal.
Normal
normal.
Normal
back.
Category is DoS: back.
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
back.
Category is DoS: back.
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
back.
Category is DoS: back.
no

normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal
normal.
Normal


In [94]:
"""Metrics to assess performance on classification task given class prediction
Functions named as ``*_score`` return a scalar value to maximize: the higher
the better
Function named as ``*_error`` or ``*_loss`` return a scalar value to minimize:
the lower the better
"""

# Authors: Alexandre Gramfort <alexandre.gramfort@inria.fr>
#          Mathieu Blondel <mathieu@mblondel.org>
#          Olivier Grisel <olivier.grisel@ensta.org>
#          Arnaud Joly <a.joly@ulg.ac.be>
#          Jochen Wersdorfer <jochen@wersdoerfer.de>
#          Lars Buitinck
#          Joel Nothman <joel.nothman@gmail.com>
#          Noel Dawe <noel@dawe.me>
#          Jatin Shah <jatindshah@gmail.com>
#          Saurabh Jha <saurabh.jhaa@gmail.com>
#          Bernardo Stein <bernardovstein@gmail.com>
#          Shangwu Yao <shangwuyao@gmail.com>
# License: BSD 3 clause


import warnings
import numpy as np

from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

from sklearn.preprocessing import LabelBinarizer, label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import assert_all_finite
from sklearn.utils import check_array
from sklearn.utils import check_consistent_length
from sklearn.utils import column_or_1d
from sklearn.utils.multiclass import unique_labels
from sklearn.utils.multiclass import type_of_target
from sklearn.utils.validation import _num_samples
from sklearn.utils.sparsefuncs import count_nonzero
from sklearn.exceptions import UndefinedMetricWarning


def _check_targets(y_true, y_pred):
    """Check that y_true and y_pred belong to the same classification task
    This converts multiclass or binary types to a common shape, and raises a
    ValueError for a mix of multilabel and multiclass targets, a mix of
    multilabel formats, for the presence of continuous-valued or multioutput
    targets, or for targets of different lengths.
    Column vectors are squeezed to 1d, while multilabel formats are returned
    as CSR sparse label indicators.
    Parameters
    ----------
    y_true : array-like
    y_pred : array-like
    Returns
    -------
    type_true : one of {'multilabel-indicator', 'multiclass', 'binary'}
        The type of the true target data, as output by
        ``utils.multiclass.type_of_target``
    y_true : array or indicator matrix
    y_pred : array or indicator matrix
    """
    check_consistent_length(y_true, y_pred)
    type_true = type_of_target(y_true)
    type_pred = type_of_target(y_pred)

    y_type = {type_true, type_pred}
    if y_type == {"binary", "multiclass"}:
        y_type = {"multiclass"}

    if len(y_type) > 1:
        raise ValueError("Classification metrics can't handle a mix of {0} "
                         "and {1} targets".format(type_true, type_pred))

    # We can't have more than one value on y_type => The set is no more needed
    y_type = y_type.pop()

    # No metrics support "multiclass-multioutput" format
    if (y_type not in ["binary", "multiclass", "multilabel-indicator"]):
        raise ValueError("{0} is not supported".format(y_type))

    if y_type in ["binary", "multiclass"]:
        y_true = column_or_1d(y_true)
        y_pred = column_or_1d(y_pred)
        if y_type == "binary":
            unique_values = np.union1d(y_true, y_pred)
            if len(unique_values) > 2:
                y_type = "multiclass"

    if y_type.startswith('multilabel'):
        y_true = csr_matrix(y_true)
        y_pred = csr_matrix(y_pred)
        y_type = 'multilabel-indicator'

    return y_type, y_true, y_pred


def _weighted_sum(sample_score, sample_weight, normalize=False):
    if normalize:
        return np.average(sample_score, weights=sample_weight)
    elif sample_weight is not None:
        return np.dot(sample_score, sample_weight)
    else:
        return sample_score.sum()


def accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    """Accuracy classification score.
    In multilabel classification, this function computes subset accuracy:
    the set of labels predicted for a sample must *exactly* match the
    corresponding set of labels in y_true.
    Read more in the :ref:`User Guide <accuracy_score>`.
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) labels.
    y_pred : 1d array-like, or label indicator array / sparse matrix
        Predicted labels, as returned by a classifier.
    normalize : bool, optional (default=True)
        If ``False``, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight : array-like of shape = [n_samples], optional
        Sample weights.
    Returns
    -------
    score : float
        If ``normalize == True``, return the fraction of correctly
        classified samples (float), else returns the number of correctly
        classified samples (int).
        The best performance is 1 with ``normalize == True`` and the number
        of samples with ``normalize == False``.
    See also
    --------
    jaccard_score, hamming_loss, zero_one_loss
    Notes
    -----
    In binary and multiclass classification, this function is equal
    to the ``jaccard_score`` function.
    Examples
    --------
    >>> from sklearn.metrics import accuracy_score
    >>> y_pred = [0, 2, 1, 3]
    >>> y_true = [0, 1, 2, 3]
    >>> accuracy_score(y_true, y_pred)
    0.5
    >>> accuracy_score(y_true, y_pred, normalize=False)
    2
    In the multilabel case with binary label indicators:
    >>> import numpy as np
    >>> accuracy_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2)))
    0.5
    """

    # Compute accuracy for each possible representation
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
    check_consistent_length(y_true, y_pred, sample_weight)
    if y_type.startswith('multilabel'):
        differing_labels = count_nonzero(y_true - y_pred, axis=1)
        score = differing_labels == 0
    else:
        score = y_true == y_pred

    return _weighted_sum(score, sample_weight, normalize)


def confusion_matrix(y_true, y_pred, labels=None, sample_weight=None):
    """Compute confusion matrix to evaluate the accuracy of a classification
    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` but
    predicted to be in group :math:`j`.
    Thus in binary classification, the count of true negatives is
    :math:`C_{0,0}`, false negatives is :math:`C_{1,0}`, true positives is
    :math:`C_{1,1}` and false positives is :math:`C_{0,1}`.
    Read more in the :ref:`User Guide <confusion_matrix>`.
    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (correct) target values.
    y_pred : array, shape = [n_samples]
        Estimated targets as returned by a classifier.
    labels : array, shape = [n_classes], optional
        List of labels to index the matrix. This may be used to reorder
        or select a subset of labels.
        If none is given, those that appear at least once
        in ``y_true`` or ``y_pred`` are used in sorted order.
    sample_weight : array-like of shape = [n_samples], optional
        Sample weights.
    Returns
    -------
    C : array, shape = [n_classes, n_classes]
        Confusion matrix
    References
    ----------
    .. [1] `Wikipedia entry for the Confusion matrix
           <https://en.wikipedia.org/wiki/Confusion_matrix>`_
           (Wikipedia and other references may use a different
           convention for axes)
    Examples
    --------
    >>> from sklearn.metrics import confusion_matrix
    >>> y_true = [2, 0, 2, 2, 0, 1]
    >>> y_pred = [0, 0, 2, 2, 0, 2]
    >>> confusion_matrix(y_true, y_pred)
    array([[2, 0, 0],
           [0, 0, 1],
           [1, 0, 2]])
    >>> y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
    >>> y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
    >>> confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
    array([[2, 0, 0],
           [0, 0, 1],
           [1, 0, 2]])
    In the binary case, we can extract true positives, etc as follows:
    >>> tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
    >>> (tn, fp, fn, tp)
    (0, 2, 1, 1)
    """
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
    if y_type not in ("binary", "multiclass"):
        raise ValueError("%s is not supported" % y_type)

    if labels is None:
        labels = unique_labels(y_true, y_pred)
    else:
        labels = np.asarray(labels)
        if np.all([l not in y_true for l in labels]):
            raise ValueError("At least one label specified must be in y_true")

    if sample_weight is None:
        sample_weight = np.ones(y_true.shape[0], dtype=np.int64)
    else:
        sample_weight = np.asarray(sample_weight)

    check_consistent_length(y_true, y_pred, sample_weight)

    n_labels = labels.size
    label_to_ind = {y: x for x, y in enumerate(labels)}
    # convert yt, yp into index
    y_pred = np.array([label_to_ind.get(x, n_labels + 1) for x in y_pred])
    y_true = np.array([label_to_ind.get(x, n_labels + 1) for x in y_true])

    # intersect y_pred, y_true with labels, eliminate items not in labels
    ind = np.logical_and(y_pred < n_labels, y_true < n_labels)
    y_pred = y_pred[ind]
    y_true = y_true[ind]
    # also eliminate weights of eliminated items
    sample_weight = sample_weight[ind]

    # Choose the accumulator dtype to always have high precision
    if sample_weight.dtype.kind in {'i', 'u', 'b'}:
        dtype = np.int64
    else:
        dtype = np.float64

    CM = coo_matrix((sample_weight, (y_true, y_pred)),
                    shape=(n_labels, n_labels), dtype=dtype,
                    ).toarray()

    return CM


def multilabel_confusion_matrix(y_true, y_pred, sample_weight=None,
                                labels=None, samplewise=False):
    """Compute a confusion matrix for each class or sample
    .. versionadded:: 0.21
    Compute class-wise (default) or sample-wise (samplewise=True) multilabel
    confusion matrix to evaluate the accuracy of a classification, and output
    confusion matrices for each class or sample.
    In multilabel confusion matrix :math:`MCM`, the count of true negatives
    is :math:`MCM_{:,0,0}`, false negatives is :math:`MCM_{:,1,0}`,
    true positives is :math:`MCM_{:,1,1}` and false positives is
    :math:`MCM_{:,0,1}`.
    Multiclass data will be treated as if binarized under a one-vs-rest
    transformation. Returned confusion matrices will be in the order of
    sorted unique labels in the union of (y_true, y_pred).
    Read more in the :ref:`User Guide <multilabel_confusion_matrix>`.
    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        of shape (n_samples, n_outputs) or (n_samples,)
        Ground truth (correct) target values.
    y_pred : 1d array-like, or label indicator array / sparse matrix
        of shape (n_samples, n_outputs) or (n_samples,)
        Estimated targets as returned by a classifier
    sample_weight : array-like of shape = (n_samples,), optional
        Sample weights
    labels : array-like
        A list of classes or column indices to select some (or to force
        inclusion of classes absent from the data)
    samplewise : bool, default=False
        In the multilabel case, this calculates a confusion matrix per sample
    Returns
    -------
    multi_confusion : array, shape (n_outputs, 2, 2)
        A 2x2 confusion matrix corresponding to each output in the input.
        When calculating class-wise multi_confusion (default), then
        n_outputs = n_labels; when calculating sample-wise multi_confusion
        (samplewise=True), n_outputs = n_samples. If ``labels`` is defined,
        the results will be returned in the order specified in ``labels``,
        otherwise the results will be returned in sorted order by default.
    See also
    --------
    confusion_matrix
    Notes
    -----
    The multilabel_confusion_matrix calculates class-wise or sample-wise
    multilabel confusion matrices, and in multiclass tasks, labels are
    binarized under a one-vs-rest way; while confusion_matrix calculates
    one confusion matrix for confusion between every two classes.
    Examples
    --------
    Multilabel-indicator case:
    >>> import numpy as np
    >>> from sklearn.metrics import multilabel_confusion_matrix
    >>> y_true = np.array([[1, 0, 1],
    ...                    [0, 1, 0]])
    >>> y_pred = np.array([[1, 0, 0],
    ...                    [0, 1, 1]])
    >>> multilabel_confusion_matrix(y_true, y_pred)
    array([[[1, 0],
            [0, 1]],
    <BLANKLINE>
           [[1, 0],
            [0, 1]],
    <BLANKLINE>
           [[0, 1],
            [1, 0]]])
    Multiclass case:
    >>> y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
    >>> y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
    >>> multilabel_confusion_matrix(y_true, y_pred,
    ...                             labels=["ant", "bird", "cat"])
    array([[[3, 1],
            [0, 2]],
    <BLANKLINE>
           [[5, 0],
            [1, 0]],
    <BLANKLINE>
           [[2, 1],
            [1, 2]]])
    """
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
    if sample_weight is not None:
        sample_weight = column_or_1d(sample_weight)
    check_consistent_length(y_true, y_pred, sample_weight)

    if y_type not in ("binary", "multiclass", "multilabel-indicator"):
        raise ValueError("%s is not supported" % y_type)

    present_labels = unique_labels(y_true, y_pred)
    if labels is None:
        labels = present_labels
        n_labels = None
    else:
        n_labels = len(labels)
        labels = np.hstack([labels, np.setdiff1d(present_labels, labels,
                                                 assume_unique=True)])

    if y_true.ndim == 1:
        if samplewise:
            raise ValueError("Samplewise metrics are not available outside of "
                             "multilabel classification.")

        le = LabelEncoder()
        le.fit(labels)
        y_true = le.transform(y_true)
        y_pred = le.transform(y_pred)
        sorted_labels = le.classes_

        # labels are now from 0 to len(labels) - 1 -> use bincount
        tp = y_true == y_pred
        tp_bins = y_true[tp]
        if sample_weight is not None:
            tp_bins_weights = np.asarray(sample_weight)[tp]
        else:
            tp_bins_weights = None

        if len(tp_bins):
            tp_sum = np.bincount(tp_bins, weights=tp_bins_weights,
                                 minlength=len(labels))
        else:
            # Pathological case
            true_sum = pred_sum = tp_sum = np.zeros(len(labels))
        if len(y_pred):
            pred_sum = np.bincount(y_pred, weights=sample_weight,
                                   minlength=len(labels))
        if len(y_true):
            true_sum = np.bincount(y_true, weights=sample_weight,
                                   minlength=len(labels))

        # Retain only selected labels
        indices = np.searchsorted(sorted_labels, labels[:n_labels])
        tp_sum = tp_sum[indices]
        true_sum = true_sum[indices]
        pred_sum = pred_sum[indices]

    else:
        sum_axis = 1 if samplewise else 0

        # All labels are index integers for multilabel.
        # Select labels:
        if not np.array_equal(labels, present_labels):
            if np.max(labels) > np.max(present_labels):
                raise ValueError('All labels must be in [0, n labels) for '
                                 'multilabel targets. '
                                 'Got %d > %d' %
                                 (np.max(labels), np.max(present_labels)))
            if np.min(labels) < 0:
                raise ValueError('All labels must be in [0, n labels) for '
                                 'multilabel targets. '
                                 'Got %d < 0' % np.min(labels))

        if n_labels is not None:
            y_true = y_true[:, labels[:n_labels]]
            y_pred = y_pred[:, labels[:n_labels]]

        # calculate weighted counts
        true_and_pred = y_true.multiply(y_pred)
        tp_sum = count_nonzero(true_and_pred, axis=sum_axis,
                               sample_weight=sample_weight)
        pred_sum = count_nonzero(y_pred, axis=sum_axis,
                                 sample_weight=sample_weight)
        true_sum = count_nonzero(y_true, axis=sum_axis,
                                 sample_weight=sample_weight)

    fp = pred_sum - tp_sum
    fn = true_sum - tp_sum
    tp = tp_sum

    if sample_weight is not None and samplewise:
        sample_weight = np.array(sample_weight)
        tp = np.array(tp)
        fp = np.array(fp)
        fn = np.array(fn)
        tn = sample_weight * y_true.shape[1] - tp - fp - fn
    elif sample_weight is not None:
        tn = sum(sample_weight) - tp - fp - fn
    elif samplewise:
        tn = y_true.shape[1] - tp - fp - fn
    else:
        tn = y_true.shape[0] - tp - fp - fn

    return np.array([tn, fp, fn, tp]).T.reshape(-1, 2, 2)


In [95]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
predictions = model2.predict(X_val)
#y_val = np.argmax(y_val, axis=-1)
predictions = np.argmax(predictions, axis=-1)
c = confusion_matrix(y_val,predictions)
#fpr, tpr, threshold = roc_curve(y_val, predictions)
print(c)
#DR=[0]*length
#FAR=[0]*length
#Efficiency=[0]*length
#for i in range(length):
    #DR[i] =c[i,0, 0] / (c[i,0, 1] + c[i,0, 0])
    #FAR[i] =c[i,0, 1] / (c[i,1, 1] + c[i,0, 1])
    #Efficiency[i]=DR[i]/FAR[i]
#print("Detection Rate is:"
      #,DR)
#print("False Alarm Rate is :"
      #,FAR)
#print("Efficiency:",Efficiency)

[[99  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  1  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0 83  0  0  0  7  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0 99  0  0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0 97  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0 91  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0]
 [59  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 70 37  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0

In [99]:
#from sklearn.preprocessing import label_binarize
#y_val = label_binarize(y_val, classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])
#n_classes = y_val.shape[1]
#print(y_val)
#print(n_classes)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
23


In [139]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(length):
    fpr[i], tpr[i], _ = roc_curve(y_val[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


IndexError: too many indices for array